In [1]:
#!/usr/bin/env python3
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from urllib.parse import urlparse

import requests
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse
import pandas as pd
import numpy as np


class Book:
    def __init__(self, name, nofeatures):
        self.name = name
        self.webpage = ""
        self.title = ""
        self.coverlink = ""
        self.keywords = []
        self.nofeatures = nofeatures

        self.get_webpage()
        self.get_keywords()
        self.unikeywords.extend(self.bikeywords)
        self.df = pd.DataFrame({'Name': [self.name], 'Keywords': [self.unikeywords]})

    def get_webpage(self):
        query = self.name + ' site:Goodreads.com'
        g_clean = []  # this is the list we store the search results
        url = 'https://www.google.com/search?client=ubuntu&channel=fs&q={}&ie=utf-8&oe=utf-8'.format(
            query)  # this is the actual query we are going to scrape

        html = requests.get(url)
        soup = BeautifulSoup(html.text)
        a = soup.find_all('a')  # a is a list
        a[0].get('href')
        for i in a:
            k = i.get('href')
            try:
                m = re.search("(?P<url>https?://[^\s]+)", k)
                n = m.group(0)
                rul = n.split('&')[0]
                domain = urlparse(rul)
                if (re.search('google.com', domain.netloc)):
                    continue
                else:
                    g_clean.append(rul)
            except:
                continue
        self.webpage = g_clean[0]

    def get_keywords(self):
        # get stopwords
        stop_nltk = set(stopwords.words("english"))
        stop_custom = {'and', 'best', 'book', 'books', 'character', 'characters', 'did', 'end', 'ending', 'film',
                       'films', 'great', 'just', 'like', 'make', 'movie', 'movies', 'novel', 'permalink', 'read',
                       'review', 'story', 'story', 'the', 'when', 'see', 'seen', 'get', 'many', 'one', 'made',
                       'ever', 'every', 'vote', 'much', 'well', 'watch', 'even', 'everything', 'youll', 'would',
                       'makes', 'even', 'ive', 'really', 'say', 'two', 'three', 'really', 'time', 'reading', 'read',
                       'first', 'going', 'good', 'little', 'new', 'things', 'thing', 'yet', 'us', 'want', 'fiction',
                       'science', 'novella', 'people', 'something', 'know', 'though', 'go', 'post', 'back', 'series'
                       'year','years', 'http', 'www'}
        stop_words = stop_nltk.union(stop_custom)

        # get book keywords and title of book, scrape from internet
        r = requests.get(self.webpage)
        html_soup = BeautifulSoup(r.content)
        self.coverlink = html_soup.find('img', id='coverImage')['src']
        self.title = html_soup.find('h1', id='bookTitle').text.strip()

        # Extract reviews and vectorize the text, returning the keywords
        review_containers = html_soup.find_all('div', class_='review')
        for i, review in enumerate(review_containers):
            review_containers[i] = review.find('span', class_='readable').text.encode("ascii","ignore").strip()
        if len(review_containers) < 2:
            return
        
    def vectorize(self, keyword, degree)
        Vect = TfidfVectorizer(max_features=self.nofeatures, ngram_range=(degree,degree), stop_words=stop_words, max_df=0.80)
        Book_Vect = Vect.fit_transform(review_containers)
        Book_feature_names = list(Vect.get_feature_names())
        keyword = Book_feature_names
        
        biVect = TfidfVectorizer(max_features=self.nofeatures, ngram_range=(2,2), stop_words=stop_words, max_df=0.80)
        biBook_Vect = biVect.fit_transform(review_containers)
        biBook_feature_names = list(biVect.get_feature_names())
        self.unikeywords = uniBook_feature_names
        self.bikeywords = biBook_feature_names

In [2]:
class Movies:
    def __init__(self,book):
        self.df = pd.read_pickle(
            '/home/colin/Insight_Project/data/smallset_2.pkl')
        self.keywords = []
        self.webpage = ""
        self.title = ""
        self.coverlink = ""
        self.second_best = ""
        self.third_best = ""
        self.combine_frames(book)
        self.best_movie()
        self.grab_movie_img()

    def combine_frames(self,book):
        Book_df = book.df
        self.df = Book_df.append(self.df).reset_index(drop=True)

    def best_movie(self):
        # Determine which movie has the most features in common with the book
        mlb = MultiLabelBinarizer()
        temp_df = self.df.copy()
        print('b4 mlb')
        clean_df = temp_df.join(pd.DataFrame(mlb.fit_transform(temp_df.pop('Keywords')),
                                   columns=mlb.classes_,
                                   index=temp_df.index))
        print('post mlb')
        sparse_df = clean_df.drop(columns='Name')
        print('pre NN')
        print(sparse_df.shape)
        nbrs = NearestNeighbors(n_neighbors=4, algorithm='ball_tree').fit(sparse_df.iloc[1:])
        distances, top_three = nbrs.kneighbors(pd.DataFrame(sparse_df.iloc[0]).T)
        print('post NN',top_three)
        self.keywords = self.df['Keywords'][top_three[0][0]+1]
        self.title = clean_df['Name'][top_three[0][0]+1]
        self.second_best =clean_df['Name'][top_three[0][1]+1]
        self.third_best = clean_df['Name'][top_three[0][2]+1]
        #return H, clean_df['Name']
        #return sparse_df

    def grab_movie_img(self):
        query = self.title + ' site:imdb.com'
        g_clean = []
        self.webpage = 'https://www.google.com/search?client=ubuntu&channel=fs&q={}&ie=utf-8&oe=utf-8'.format(
            query)  # this is the actual query we are going to scrape
        html = requests.get(self.webpage)
        soup = BeautifulSoup(html.text)
        a = soup.find_all('a')  # a is a list
        a[0].get('href')
        for i in a:
            k = i.get('href')
            try:
                m = re.search("(?P<url>https?://[^\s]+)", k)
                n = m.group(0)
                rul = n.split('&')[0]
                domain = urlparse(rul)
                if re.search("google.com", domain.netloc):
                    continue
                else:
                    g_clean.append(rul)
            except:
                continue
        url = g_clean[0]
        html = requests.get(url)
        soup = BeautifulSoup(html.text)
        self.coverlink = soup.find(class_='poster').img['src']
        
        

In [3]:
test = Book('oryx and crake',50)

In [5]:
test.keywords()

AttributeError: 'Book' object has no attribute 'keywords'

In [1]:
x = [[]]

In [3]:
x.append([])